.. _nb_repair:

## Repair Operator

The repair operator is mostly problem dependent. Most commonly it is used to make sure the algorithm is only searching in the feasible space. It is applied after the offsprings have been reproduced. In the following, we are using the knapsack problem to demonstrate the repair operator in *pymoo*.



In the well-known **Knapsack Problem**. In this problem, a knapsack has to be filled with items without violating the maximum weight constraint. Each item $j$ has a value $b_j \geq 0$  and a weight $w_j \geq 0$ where $j \in \{1, .., m\}$. The binary decision vector $z = (z_1, .., z_m)$ defines, if an item is picked or not. The aim is to maximize the profit $g(z)$:

\begin{eqnarray}
max & & g(z) \\[2mm] \notag 
\text{s.t.} & & \sum_{j=1}^m z_j \, w_j \leq Q \\[1mm] \notag 
& & z = (z_1, .., z_m) \in \mathbb{B}^m \\[1mm] \notag 
g(z) & = & \sum_{j=1}^{m}  z_j \, b_j \\[2mm] \notag 
\end{eqnarray}


A simple GA will have some infeasible evaluations in the beginning and then concentrate on the infeasible space.


In [1]:
from pymoo.factory import get_algorithm, get_crossover, get_mutation, get_sampling
from pymoo.optimize import minimize
from pymoo.problems.single.knapsack import create_random_knapsack_problem

problem = create_random_knapsack_problem(30)

algorithm = get_algorithm("ga",
                       pop_size=200,
                       sampling=get_sampling("bin_random"),
                       crossover=get_crossover("bin_hux"),
                       mutation=get_mutation("bin_bitflip"),
                       eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               termination=('n_gen', 10),
               verbose=True)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |     favg     |     fopt    
    1 |     200 |  2.36000E+02 |  5.18510E+02 |            - |            -
    2 |     400 |  8.40000E+01 |  3.83695E+02 |            - |            -
    3 |     600 |  8.40000E+01 |  2.87015E+02 |            - |            -
    4 |     800 |  8.000000000 |  2.10900E+02 |            - |            -
    5 |    1000 |  0.00000E+00 |  1.45015E+02 | -2.48000E+02 |         -264
    6 |    1200 |  0.00000E+00 |  8.50150E+01 | -2.64267E+02 |         -419
    7 |    1400 |  0.00000E+00 |  3.86550E+01 | -2.84909E+02 |         -686
    8 |    1600 |  0.00000E+00 |  9.655000000 | -2.92924E+02 |         -686
    9 |    1800 |  0.00000E+00 |  0.070000000 | -2.99677E+02 |         -686
   10 |    2000 |  0.00000E+00 |  0.00000E+00 | -3.73755E+02 |         -686



Because the constraint $\sum_{j=1}^m z_j \, w_j \leq Q$ is fairly easy to satisfy. Therefore, we can make sure before evaluating the objective function, that this constraint is not violated by repairing the individual.
A repair class has to be defined and the population are given as input. The repaired population has to be returned.

In [2]:
import numpy as np
from pymoo.model.repair import Repair


class ConsiderMaximumWeightRepair(Repair):

    def _do(self, problem, pop, **kwargs):
        
        # maximum capacity for the problem
        Q = problem.C
        
        # the packing plan for the whole population (each row one individual)
        Z = pop.get("X")
        
        # the corresponding weight of each individual
        weights = (Z * problem.W).sum(axis=1)
        
        # now repair each indvidiual i
        for i in range(len(Z)):
            
            # the packing plan for i
            z = Z[i]
            
            # while the maximum capacity violation holds
            while weights[i] > Q:
                
                # randomly select an item currently picked
                item_to_remove = np.random.choice(np.where(z)[0])
                
                # and remove it
                z[item_to_remove] = False
                
                # adjust the weight
                weights[i] -= problem.W[item_to_remove]
          
        # set the design variables for the population
        pop.set("X", Z)
        return pop

In [3]:
algorithm.repair = ConsiderMaximumWeightRepair()

res = minimize(problem,
               algorithm,
               termination=('n_gen', 10),
               verbose=True)


n_gen |  n_eval |   cv (min)   |   cv (avg)   |     favg     |     fopt    
    1 |     200 |  0.00000E+00 |  0.00000E+00 | -1.34325E+02 |         -387
    2 |     400 |  0.00000E+00 |  0.00000E+00 | -2.23300E+02 |         -387
    3 |     600 |  0.00000E+00 |  0.00000E+00 | -2.80105E+02 |         -412
    4 |     800 |  0.00000E+00 |  0.00000E+00 | -3.30060E+02 |         -542
    5 |    1000 |  0.00000E+00 |  0.00000E+00 | -3.76335E+02 |         -544
    6 |    1200 |  0.00000E+00 |  0.00000E+00 | -4.16160E+02 |         -579
    7 |    1400 |  0.00000E+00 |  0.00000E+00 | -4.50245E+02 |         -610


    8 |    1600 |  0.00000E+00 |  0.00000E+00 | -4.85835E+02 |         -648
    9 |    1800 |  0.00000E+00 |  0.00000E+00 | -5.19985E+02 |         -648
   10 |    2000 |  0.00000E+00 |  0.00000E+00 | -5.43220E+02 |         -670


As it can be seen, the repair operator makes sure no infeasible solution is evaluated. Even though this example seems to be quite easy, the repair operator makes especially sense for more complex constraints where domain specific knowledge is known.